<a href="https://colab.research.google.com/github/nadia-azam/NLP_TP/blob/main/TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install kaggle

In [15]:
from google.colab import files

# Sélectionnez le fichier kaggle.json que vous avez téléchargé
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"azamnadia","key":"8a7c9d24fa321d9294b1762e4daac842"}'}

In [16]:
# creating a kaggle folder
!mkdir -p ~/.kaggle

In [17]:
# copy kaggle.json to the folder created
!cp kaggle.json ~/.kaggle/

In [18]:
!chmod 600 ~/.kaggle/kaggle.json

In [19]:
! kaggle datasets list

ref                                                    title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                     Vehicle Sales Data                               19MB  2024-02-21 20:16:17          16735        295  1.0              
sudarshan24byte/online-food-dataset                    Online Food Dataset                               3KB  2024-03-02 18:50:30           6272        113  0.9411765        
nbroad/gemma-rewrite-nbroad                            gemma-rewrite-nbroad                              8MB  2024-03-03 04:52:39            782         69  1.0              
arnavvvvv/spotify-music                                Top Spotify Songs                                47KB  2024-03-06 05:2

In [20]:
!kaggle datasets download -d nltkdata/movie-review

movie-review.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
import zipfile

with zipfile.ZipFile("movie-review.zip", "r") as zip_ref:
    zip_ref.extractall("dataset")

In [22]:
# chargement des librairies :
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [23]:
# Chargez les données
data = pd.read_csv("/content/dataset/movie_review.csv")


In [25]:
data

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [26]:
data.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


In [27]:
#Pre-processing des données textuelles :
# Téléchargez les stopwords de NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [28]:
# Fonction pour nettoyer et prétraiter le texte
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation et les caractères spéciaux
    text = re.sub(r'\W', ' ', text)
    # Supprimer les stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

In [29]:
# Appliquer la fonction de prétraitement aux données textuelles
data['text'] = data['text'].apply(preprocess_text)

In [30]:
data.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted comic books plenty success wheth...,pos
1,0,cv000,29590,1,starters created alan moore eddie campbell bro...,pos
2,0,cv000,29590,2,say moore campbell thoroughly researched subje...,pos
3,0,cv000,29590,3,book graphic novel 500 pages long includes nea...,pos
4,0,cv000,29590,4,words dismiss film source,pos


In [31]:
#  Entraînement du modèle Word2Vec

In [33]:
# Tokenisation des textes prétraités
tokenized_text = [text.split() for text in data['text']]

In [34]:
tokenized_text

[['films',
  'adapted',
  'comic',
  'books',
  'plenty',
  'success',
  'whether',
  'superheroes',
  'batman',
  'superman',
  'spawn',
  'geared',
  'toward',
  'kids',
  'casper',
  'arthouse',
  'crowd',
  'ghost',
  'world',
  'never',
  'really',
  'comic',
  'book',
  'like',
  'hell'],
 ['starters',
  'created',
  'alan',
  'moore',
  'eddie',
  'campbell',
  'brought',
  'medium',
  'whole',
  'new',
  'level',
  'mid',
  '80s',
  '12',
  'part',
  'series',
  'called',
  'watchmen'],
 ['say',
  'moore',
  'campbell',
  'thoroughly',
  'researched',
  'subject',
  'jack',
  'ripper',
  'would',
  'like',
  'saying',
  'michael',
  'jackson',
  'starting',
  'look',
  'little',
  'odd'],
 ['book',
  'graphic',
  'novel',
  '500',
  'pages',
  'long',
  'includes',
  'nearly',
  '30',
  'consist',
  'nothing',
  'footnotes'],
 ['words', 'dismiss', 'film', 'source'],
 ['get',
  'past',
  'whole',
  'comic',
  'book',
  'thing',
  'might',
  'find',
  'another',
  'stumbling',
  

In [35]:
# Entraînement du modèle Word2Vec
word2vec_model = Word2Vec(tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

In [36]:
# Exemple d'utilisation
word2vec_model.wv['movie']  # Vecteur pour le mot "movie"

array([-0.72289115,  1.0409307 ,  0.5499595 ,  0.69348925, -0.17207934,
       -0.94242823,  1.3369344 ,  1.6874174 , -0.8319964 , -0.41503733,
       -0.6521823 , -1.5351838 ,  0.42101794,  0.8894418 ,  0.4090222 ,
       -0.19501758,  0.0600054 ,  0.04528978, -1.0647364 , -1.2839833 ,
       -0.33873552,  1.0007277 ,  0.905263  , -0.0710886 ,  1.0136154 ,
       -0.09999987, -0.6935443 ,  0.01405531, -1.2018979 ,  0.13388073,
        0.38933492, -0.25193742,  0.12425616, -1.4589634 , -0.8646161 ,
        1.0553955 ,  1.3232399 , -1.1573873 , -0.29587147, -0.9132547 ,
       -0.50183946, -0.7227968 , -1.4252771 ,  0.37619385,  0.5783799 ,
       -0.2948069 , -0.5675219 , -0.39421022,  0.47426856,  0.044416  ,
       -0.06030497, -0.5363645 , -0.7052962 ,  0.6493547 , -0.31749037,
       -0.4067024 ,  0.6239978 , -0.8531927 , -0.21884505,  0.45126286,
       -0.0225096 ,  0.13398015,  0.7913482 , -0.54756784, -0.73280513,
        1.2628167 , -0.15826498,  1.0926162 , -1.2952547 ,  0.76

In [37]:
#  Vectorisation des reviews de movies

In [39]:
# Fonction pour calculer la moyenne des embeddings Word2Vec pour chaque review
"""Args:
    - words (list): La liste des mots de la review.
    - model: Le modèle Word2Vec entraîné.
    - vocabulary (set): Le vocabulaire du modèle Word2Vec.
    - num_features (int): Le nombre de features dans chaque embedding.
            le nombre de dimensions dans les embeddings Word2Vec
    et return : - feature_vector (numpy.ndarray): Le vecteur de features moyen pour la review.
    """

def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.

    for word in words:
        if word in vocabulary:
            nwords = nwords + 1
            feature_vector = np.add(feature_vector, model.wv[word])

    if nwords:
        feature_vector = np.divide(feature_vector, nwords)

    return feature_vector

# Construire le vocabulaire
vocabulary = set(word2vec_model.wv.index_to_key)

# Créer les vecteurs de features pour chaque review
data['text'] = [average_word_vectors(text, word2vec_model, vocabulary, 100) for text in tokenized_text]


In [41]:
# Division des données en ensembles d'entraînement et de test
""" ne pas charger un x_train a chaque fois on execute """

X_train, X_test, y_train, y_test = train_test_split(data['text'], data['tag'], test_size=0.2, random_state=42)

In [42]:
X_train

48063    [-0.11205093402947698, 0.512106291949749, 0.39...
7748     [-0.05208041770383716, 0.35136730992235243, 0....
39824    [-0.1408871879801154, 0.5143764764070511, 0.39...
49957    [-0.10974748634422819, 0.385332927107811, 0.27...
33990    [-0.17068151946295984, 0.5165817671222612, 0.3...
                               ...                        
62570    [-0.10618587468257722, 0.4998688826008755, 0.3...
38158    [-0.2372949942946434, 0.6433608569204807, 0.31...
860      [-0.08729307121959412, 0.4516808393721779, 0.4...
15795    [-0.13690912094898522, 0.4388743175400628, 0.3...
56422    [-0.0824502354177336, 0.4515733062511399, 0.30...
Name: text, Length: 51776, dtype: object

In [49]:
# Construction d'un classificateur :

In [45]:
# Initialisation du modèle de classification (Exemple avec Logistic Regression)
classifier = LogisticRegression(max_iter=1000)

# Entraînement du modèle
classifier.fit(list(X_train), y_train)

LogisticRegression(max_iter=1000)

In [46]:
 # Évaluation du modèle

In [47]:
# Prédiction sur l'ensemble de test
y_pred = classifier.predict(list(X_test))

In [48]:
# Calcul des métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)     # nombre de prédictions correctes parmi toutes les prédictions faites par le modèle
precision = precision_score(y_test, y_pred, average='weighted')   #l'exactitude des prédictions positives du modèle
recall = recall_score(y_test, y_pred, average='weighted')   #  la capacité du modèle à identifier tous les exemples positifs
f1 = f1_score(y_test, y_pred, average='weighted')   # une mesure qui combine à la fois la précision et le rappel en calculant leur moyenne harmonique.

""" average='weighted' indique que la précision est calculée en prenant en compte le nombre d'échantillons pour chaque classe. """

# Affichage des résultats
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5761742892459827
Precision: 0.5775199483740883
Recall: 0.5761742892459827
F1 Score: 0.5720675497270609
